## Imports

In [1]:
import sys 
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline  

from sklearn import svm, linear_model
from sklearn.linear_model import SGDClassifier
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.externals import joblib

import torch
import torchvision 
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import Dataset, DataLoader

sys.path.insert(0, '../../../Utils/')
sys.path.insert(0, '../../../')
import cyphercat as cc

import models
from train import *
from metrics import * 
from SVC_Utils import *

#audio
import librosa as libr

print("Python: %s" % sys.version)
print("Pytorch: %s" % torch.__version__)

# determine device to run network on (runs on gpu if available)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


Python: 3.6.5 (default, Jul  6 2018, 19:12:46) 
[GCC 5.4.0 20160609]
Pytorch: 0.4.0


## NN Hyperparameters

In [2]:
batch_size = 128
lr = 0.001
k = 3

pretrained = False #run this with networks that have already been trained

transform_type = 'SFTF' #either STFT or MFCC  

data = 'Libri' #'Libri' or 'VOiCES'

In [10]:
### Speech preprocessing

class tensorToMFCC:
    def __call__(self, y):
#         y = y.numpy()
        dims = y.shape
        y = libr.feature.melspectrogram(np.reshape(y, (dims[1],)), 16000, n_mels=number_of_mels,
                               fmax=8000)
        y = libr.feature.mfcc(S = libr.power_to_db(y))
        y = torch.from_numpy(y)                           
        return y.float()

class STFT:
    def __call__(self,y):
        dims = y.shape
        y = np.abs(libr.core.stft(np.reshape(y, (dims[1],))))
        y = torch.from_numpy(y).permute(1,0)
        return y.float()

if transform_type == 'SFTF':
    target_net_type = cc.ft_cnn_classifer
    shadow_net_type = cc.ft_cnn_classifer
    in_size = 94# 20 forMFCC,  94 for STFT
    transform  = STFT() ## STFT or MFCC
elif transform_type == 'MFCC':
    transform  = tensorToMFCC()
    target_net_type = cc.MFCC_cnn_classifier
    shadow_net_type = cc.MFCC_cnn_classifier
    in_size = 20

## Audio hyperparameters

In [11]:
n_seconds = 3
n_epochs = 15
sampling_rate = 16000
number_of_mels =128
lr = 0.001

# attacking means data for a target & shadow network.
# This will also split "out data" from totally different speakers -- data none of the 
# other networks have seen, for training & testing the attack network. This will be
# an equivalent amount of data to the train split as defined about

%load_ext autoreload
%autoreload 2
sys.path.insert(0, './../../../Utils')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load audio data: VOiCES or LibriSpeech, & Split into valid sequences

In [12]:
print('Loading splits')

if data == 'Libri':
    dfs = cc.Libri_preload_and_split()
    print('Initializing dataset')
    valid_sequence_train_target = cc.LibriSpeechDataset(df=dfs[0], transform = transform)
    valid_sequence_test_target = cc.LibriSpeechDataset(df=dfs[1], transform = transform)
    valid_sequence_train_shadow = cc.LibriSpeechDataset(df=dfs[2], transform = transform)
    valid_sequence_test_shadow = cc.LibriSpeechDataset(df=dfs[3], transform = transform)
    valid_sequence_test = cc.LibriSpeechDataset(df=dfs[4], transform = transform)

    print('Succesfully loaded libri-speech')
elif data == 'VOiCES':
    print('Loading splits')
    dfs = cc.Voices_preload_and_split()
    print('Initializing dataset')
    valid_sequence_train_target = cc.Voices_dataset(df=dfs[0], transform = transform)
    valid_sequence_test_target = cc.Voices_dataset(df=dfs[1], transform = transform)
    valid_sequence_train_shadow = cc.Voices_dataset(df=dfs[2], transform = transform)
    valid_sequence_test_shadow = cc.Voices_dataset(df=dfs[3], transform = transform)
    valid_sequence_test = cc.Voices_dataset(df=dfs[4], transform = transform)

Loading splits
Initialising LibriSpeechDataset with minimum length = 3s and subset = train-clean-100
Finished indexing data. 27949 usable files found.
Found default splits, loading dataframe
		 ---- Split 0 ---- 
	Unique speakers 	 Samples
Male:		 63		 5539
Female:		 62		 5573
Total:		 125		 11112
		 ---- Split 1 ---- 
	Unique speakers 	 Samples
Male:		 63		 1414
Female:		 62		 1427
Total:		 125		 2841
		 ---- Split 2 ---- 
	Unique speakers 	 Samples
Male:		 63		 3471
Female:		 63		 3490
Total:		 126		 6961
		 ---- Split 3 ---- 
	Unique speakers 	 Samples
Male:		 63		 3510
Female:		 63		 3525
Total:		 126		 7035
		 ---- Split 4 ---- 
	Unique speakers 	 Samples
Male:		 16		 1414
Female:		 16		 1427
Total:		 32		 2841
Finished splitting data.
Initializing dataset
Succesfully loaded libri-speech


In [13]:
batch_size = 32


# Loaders for data for target model & shadow model 
target_train_loader = DataLoader(valid_sequence_train_target,
                      batch_size=batch_size,
                      shuffle=True,
                      num_workers=8,
                    drop_last = True
                     # pin_memory=True # CUDA only
                     )


target_test_loader = DataLoader(valid_sequence_test_target,
                      batch_size=batch_size,
                      shuffle=True,
                      num_workers=8
                     # pin_memory=True # CUDA only
                     )

shadow_train_loader = DataLoader(valid_sequence_train_shadow,
                      batch_size=batch_size,
                      shuffle=True,
                      num_workers=8,
                    drop_last = True
                     # pin_memory=True # CUDA only
                     )


shadow_test_loader = DataLoader(valid_sequence_test_shadow,
                      batch_size=batch_size,
                      shuffle=True,
                      num_workers=8
                     # pin_memory=True # CUDA only
                     )


test_loader = DataLoader(valid_sequence_test,
                      batch_size=batch_size,
                      shuffle=True,
                      num_workers=8
                     # pin_memory=True # CUDA only
                     )


## Set up model tracking

In [7]:
#table with summary

# To do: extract accuracy from train/eval funcs and automatically add to table
summary_file = 'summary.pkl'
columns = ['Transform','Training epochs', '# speakers','Train accuracy', 'Test accuracy', 'Attack type', 'Precision','Recall']

try:
    df = pd.read_pickle(summary_file)

except:
    df = pd.DataFrame(columns = columns)
    
df_idx = len(df)

#set a bunch of known values
df.at[df_idx,'Transform'] =transform_type
df.at[df_idx,'Training epochs'] = n_epochs
df.at[df_idx,'Attack type'] = 'mlleaks1'

# Initialize/Train Targets
The model being attacked; if network, architecture can differ from that of shadow network.

In [8]:
#Initialize NN

#in_size defined above
n_hidden = 512
n_classes = valid_sequence_test.num_speakers
print(n_classes,' speakers')
df.at[df_idx,'# speakers']=n_classes


target_net = target_net_type(n_classes).to(device)
target_net.apply(models.weights_init)

target_loss = nn.CrossEntropyLoss()
target_optim = optim.Adam(target_net.parameters(), lr=lr)




125  speakers


In [9]:
#file name for this set of hyperparameters
fn = 'model_weights/CNN_voice_classifier'+data+'_target_'+transform_type+str(n_epochs-1)+'.pth'

#Train NN
if not pretrained:
    train_accuracy, test_accuracy = cc.train(target_net, target_train_loader, target_test_loader, target_optim, target_loss, n_epochs, verbose = False) 
    df.at[df_idx,'Train accuracy'] =train_accuracy
    df.at[df_idx,'Test accuracy'] = test_accuracy
    cc.save_checkpoint(model = target_net, optimizer = target_optim,
                           epoch = n_epochs-1, data_descriptor = data, 
                           accuracy = [train_accuracy, test_accuracy],
                           filename = fn)
    
else:
    cc.load_checkpoint(model = target_net, optimizer = target_optim, checkpoint = fn)


[0/15]
Training:

Accuracy = 66.52 %%


Test:

Accuracy = 57.87 %%


[1/15]
Training:

Accuracy = 84.74 %%


Test:

Accuracy = 74.23 %%


[2/15]
Training:

Accuracy = 88.36 %%


Test:

Accuracy = 79.02 %%


[3/15]
Training:

Accuracy = 93.47 %%


Test:

Accuracy = 87.19 %%


[4/15]
Training:

Accuracy = 93.68 %%


Test:

Accuracy = 84.30 %%


[5/15]
Training:

Accuracy = 94.91 %%


Test:

Accuracy = 87.40 %%


[6/15]
Training:

Accuracy = 94.79 %%


Test:

Accuracy = 87.36 %%


[7/15]
Training:

Accuracy = 95.46 %%


Test:

Accuracy = 89.02 %%


[8/15]
Training:

Accuracy = 97.20 %%


Test:

Accuracy = 90.85 %%


[9/15]
Training:

Accuracy = 97.12 %%


Test:

Accuracy = 91.06 %%


[10/15]
Training:

Accuracy = 96.26 %%


Test:

Accuracy = 89.62 %%


[11/15]
Training:

Accuracy = 96.73 %%


Test:

Accuracy = 90.74 %%


[12/15]
Training:

Accuracy = 97.27 %%


Test:

Accuracy = 92.19 %%


[13/15]
Training:

Accuracy = 96.97 %%


Test:

Accuracy = 91.31 %%


[14/15]
Training:

Accuracy = 

# Initialize/Train Shadow Model
Shadow model mimics the target network, emulating the target model's differences in prediction probabilities for samples in and out of its dataset. For this attack, only one shadow model is used. 

In [19]:
#Initialize models

n_classes = valid_sequence_test_shadow.num_speakers
print('n shadow speakers',n_classes)

#NN
shadow_net = shadow_net_type(n_classes).to(device)
shadow_net.apply(models.weights_init)

shadow_loss = nn.CrossEntropyLoss()
shadow_optim = optim.Adam(shadow_net.parameters(), lr=lr)

n shadow speakers 126


# Initialize Attack Model
A binary classifier to determine membership. 

In [ ]:
# Attack the network: 

attack_net_nn = models.mlleaks_mlp(n_in=k).to(device)
attack_loss = nn.BCEWithLogitsLoss()
attack_optim_nn= optim.Adam(attack_net_nn.parameters(), lr=lr)
n_epochs_attack = 50

cc.ml_leaks1(target=target_net, shadow_model = shadow_net, attacker_model = attack_net_nn,
            target_in_loader = test_loader, target_out_loader = target_test_loader,
            shadow_train_loader = shadow_train_loader, shadow_out_loader=shadow_test_loader,
            shadow_optim = shadow_optim, attack_optim = attack_optim_nn, 
            shadow_criterion = shadow_loss, attack_criterion = attack_loss, 
            n_epochs = n_epochs_attack, retrain = True)
attack_net_nn.apply(models.weights_init)

---- Training shadow network ----
[0/50]
Training:

Accuracy = 72.61 %%


Test:

Accuracy = 55.38 %%


[1/50]
Training:

Accuracy = 80.60 %%


Test:

Accuracy = 61.25 %%


[2/50]
Training:

Accuracy = 87.57 %%


Test:

Accuracy = 67.87 %%


[3/50]
Training:

Accuracy = 89.78 %%


Test:

Accuracy = 70.36 %%


[4/50]
Training:

Accuracy = 92.04 %%


Test:

Accuracy = 73.56 %%


[5/50]
Training:

Accuracy = 91.86 %%


Test:

Accuracy = 74.03 %%


[6/50]
Training:

Accuracy = 94.11 %%


Test:

Accuracy = 74.13 %%


[7/50]
Training:

Accuracy = 94.71 %%


Test:

Accuracy = 76.52 %%


[8/50]
Training:

Accuracy = 93.82 %%


Test:

Accuracy = 75.99 %%


[9/50]
Training:

Accuracy = 96.30 %%


Test:

Accuracy = 76.84 %%


[10/50]
Training:

Accuracy = 96.11 %%


Test:

Accuracy = 77.75 %%


[11/50]
Training:

Accuracy = 96.83 %%


Test:

Accuracy = 79.56 %%


[12/50]
Training:

Accuracy = 96.00 %%


Test:

Accuracy = 78.05 %%


[13/50]
Training:

Accuracy = 95.97 %%


Test:

Accuracy = 77.07 %

[0/50][104/217] loss = 0.69, accuracy = 55.30
[0/50][105/217] loss = 0.69, accuracy = 55.42
[0/50][106/217] loss = 0.69, accuracy = 55.49
[0/50][107/217] loss = 0.69, accuracy = 55.54
[0/50][108/217] loss = 0.69, accuracy = 55.60
[0/50][109/217] loss = 0.69, accuracy = 55.67
[0/50][110/217] loss = 0.69, accuracy = 55.74
[0/50][111/217] loss = 0.69, accuracy = 55.87
[0/50][112/217] loss = 0.69, accuracy = 55.93
[0/50][113/217] loss = 0.69, accuracy = 55.89
[0/50][114/217] loss = 0.69, accuracy = 55.99
[0/50][115/217] loss = 0.69, accuracy = 55.98
[0/50][116/217] loss = 0.69, accuracy = 56.00
[0/50][117/217] loss = 0.69, accuracy = 56.01
[0/50][118/217] loss = 0.69, accuracy = 56.08
[0/50][119/217] loss = 0.69, accuracy = 56.11
[0/50][120/217] loss = 0.69, accuracy = 56.13
[0/50][121/217] loss = 0.69, accuracy = 56.16
[0/50][122/217] loss = 0.69, accuracy = 56.16
[0/50][123/217] loss = 0.69, accuracy = 56.21
[0/50][124/217] loss = 0.69, accuracy = 56.24
[0/50][125/217] loss = 0.69, accur

[1/50][68/217] loss = 0.68, accuracy = 59.90
[1/50][69/217] loss = 0.67, accuracy = 59.98
[1/50][70/217] loss = 0.68, accuracy = 60.04
[1/50][71/217] loss = 0.68, accuracy = 60.00
[1/50][72/217] loss = 0.68, accuracy = 59.97
[1/50][73/217] loss = 0.68, accuracy = 59.92
[1/50][74/217] loss = 0.67, accuracy = 59.94
[1/50][75/217] loss = 0.68, accuracy = 59.93
[1/50][76/217] loss = 0.67, accuracy = 59.94
[1/50][77/217] loss = 0.67, accuracy = 60.00
[1/50][78/217] loss = 0.68, accuracy = 60.01
[1/50][79/217] loss = 0.68, accuracy = 60.02
[1/50][80/217] loss = 0.66, accuracy = 60.11
[1/50][81/217] loss = 0.69, accuracy = 60.04
[1/50][82/217] loss = 0.68, accuracy = 59.94
[1/50][83/217] loss = 0.68, accuracy = 59.91
[1/50][84/217] loss = 0.69, accuracy = 59.85
[1/50][85/217] loss = 0.67, accuracy = 59.90
[1/50][86/217] loss = 0.67, accuracy = 59.91
[1/50][87/217] loss = 0.68, accuracy = 59.91
[1/50][88/217] loss = 0.67, accuracy = 59.97
[1/50][89/217] loss = 0.69, accuracy = 59.95
[1/50][90/

Exception ignored in: <bound method _DataLoaderIter.__del__ of <torch.utils.data.dataloader._DataLoaderIter object at 0x7fa4c7126160>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 349, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 328, in _shutdown_workers
    self.worker_result_queue.get()
  File "/usr/local/lib/python3.6/multiprocessing/queues.py", line 337, in get
    return _ForkingPickler.loads(res)
  File "/usr/local/lib/python3.6/site-packages/torch/multiprocessing/reductions.py", line 70, in rebuild_storage_fd
    fd = df.detach()
  File "/usr/local/lib/python3.6/multiprocessing/resource_sharer.py", line 58, in detach
    return reduction.recv_handle(conn)
  File "/usr/local/lib/python3.6/multiprocessing/reduction.py", line 182, in recv_handle
    return recvfds(s, 1)[0]
  File "/usr/local/lib/python3.6/multiprocessing/reducti

[2/50][0/217] loss = 0.66, accuracy = 59.38
[2/50][1/217] loss = 0.65, accuracy = 61.72
[2/50][2/217] loss = 0.67, accuracy = 61.46
[2/50][3/217] loss = 0.67, accuracy = 60.55
[2/50][4/217] loss = 0.64, accuracy = 61.88
[2/50][5/217] loss = 0.67, accuracy = 61.46
[2/50][6/217] loss = 0.68, accuracy = 60.27
[2/50][7/217] loss = 0.71, accuracy = 59.77
[2/50][8/217] loss = 0.64, accuracy = 59.90
[2/50][9/217] loss = 0.68, accuracy = 59.53
[2/50][10/217] loss = 0.72, accuracy = 58.81
[2/50][11/217] loss = 0.68, accuracy = 58.20
[2/50][12/217] loss = 0.67, accuracy = 58.05
[2/50][13/217] loss = 0.68, accuracy = 57.92
[2/50][14/217] loss = 0.64, accuracy = 58.12
[2/50][15/217] loss = 0.66, accuracy = 58.40
[2/50][16/217] loss = 0.66, accuracy = 58.36
[2/50][17/217] loss = 0.68, accuracy = 58.59
[2/50][18/217] loss = 0.66, accuracy = 58.96
[2/50][19/217] loss = 0.68, accuracy = 58.67
[2/50][20/217] loss = 0.65, accuracy = 58.93
[2/50][21/217] loss = 0.65, accuracy = 59.09
[2/50][22/217] loss 

...

[29/50][2/217] loss = 0.71, accuracy = 55.21
[29/50][3/217] loss = 0.65, accuracy = 56.64
[29/50][4/217] loss = 0.66, accuracy = 56.88
[29/50][5/217] loss = 0.64, accuracy = 57.81
[29/50][6/217] loss = 0.67, accuracy = 57.59
[29/50][7/217] loss = 0.69, accuracy = 57.62
[29/50][8/217] loss = 0.69, accuracy = 58.16
[29/50][9/217] loss = 0.69, accuracy = 57.97
[29/50][10/217] loss = 0.62, accuracy = 58.38
[29/50][11/217] loss = 0.67, accuracy = 58.07
[29/50][12/217] loss = 0.67, accuracy = 58.17
[29/50][13/217] loss = 0.63, accuracy = 58.59
[29/50][14/217] loss = 0.58, accuracy = 59.38
[29/50][15/217] loss = 0.66, accuracy = 59.47
[29/50][16/217] loss = 0.71, accuracy = 59.56
[29/50][17/217] loss = 0.69, accuracy = 59.64
[29/50][18/217] loss = 0.65, accuracy = 59.62
[29/50][19/217] loss = 0.69, accuracy = 59.38
[29/50][20/217] loss = 0.63, accuracy = 59.60
[29/50][21/217] loss = 0.61, accuracy = 59.73
[29/50][22/217] loss = 0.67, accuracy = 59.78
[29/50][23/217] loss = 0.67, accuracy = 59

[29/50][179/217] loss = 0.60, accuracy = 60.81
[29/50][180/217] loss = 0.70, accuracy = 60.79
[29/50][181/217] loss = 0.62, accuracy = 60.82
[29/50][182/217] loss = 0.64, accuracy = 60.82
[29/50][183/217] loss = 0.65, accuracy = 60.81
[29/50][184/217] loss = 0.70, accuracy = 60.77
[29/50][185/217] loss = 0.67, accuracy = 60.79
[29/50][186/217] loss = 0.61, accuracy = 60.81
[29/50][187/217] loss = 0.63, accuracy = 60.82
[29/50][188/217] loss = 0.62, accuracy = 60.82
[29/50][189/217] loss = 0.62, accuracy = 60.83
[29/50][190/217] loss = 0.63, accuracy = 60.83
[29/50][191/217] loss = 0.64, accuracy = 60.83
[29/50][192/217] loss = 0.66, accuracy = 60.81
[29/50][193/217] loss = 0.67, accuracy = 60.79
[29/50][194/217] loss = 0.62, accuracy = 60.82
[29/50][195/217] loss = 0.68, accuracy = 60.78
[29/50][196/217] loss = 0.62, accuracy = 60.79
[29/50][197/217] loss = 0.65, accuracy = 60.79
[29/50][198/217] loss = 0.67, accuracy = 60.75
[29/50][199/217] loss = 0.64, accuracy = 60.76
[29/50][200/2

[30/50][141/217] loss = 0.70, accuracy = 60.34
[30/50][142/217] loss = 0.66, accuracy = 60.31
[30/50][143/217] loss = 0.64, accuracy = 60.31
[30/50][144/217] loss = 0.64, accuracy = 60.32
[30/50][145/217] loss = 0.64, accuracy = 60.33
[30/50][146/217] loss = 0.66, accuracy = 60.29
[30/50][147/217] loss = 0.67, accuracy = 60.26
[30/50][148/217] loss = 0.68, accuracy = 60.27
[30/50][149/217] loss = 0.76, accuracy = 60.22
[30/50][150/217] loss = 0.70, accuracy = 60.24
[30/50][151/217] loss = 0.62, accuracy = 60.26
[30/50][152/217] loss = 0.61, accuracy = 60.27
[30/50][153/217] loss = 0.59, accuracy = 60.32
[30/50][154/217] loss = 0.73, accuracy = 60.26
[30/50][155/217] loss = 0.62, accuracy = 60.27
[30/50][156/217] loss = 0.60, accuracy = 60.29
[30/50][157/217] loss = 0.68, accuracy = 60.25
[30/50][158/217] loss = 0.61, accuracy = 60.29
[30/50][159/217] loss = 0.65, accuracy = 60.29
[30/50][160/217] loss = 0.69, accuracy = 60.25
[30/50][161/217] loss = 0.64, accuracy = 60.25
[30/50][162/2

[31/50][102/217] loss = 0.73, accuracy = 60.30
[31/50][103/217] loss = 0.68, accuracy = 60.35
[31/50][104/217] loss = 0.61, accuracy = 60.42
[31/50][105/217] loss = 0.64, accuracy = 60.44
[31/50][106/217] loss = 0.67, accuracy = 60.40
[31/50][107/217] loss = 0.67, accuracy = 60.36
[31/50][108/217] loss = 0.65, accuracy = 60.34
[31/50][109/217] loss = 0.62, accuracy = 60.37
[31/50][110/217] loss = 0.67, accuracy = 60.33
[31/50][111/217] loss = 0.66, accuracy = 60.31
[31/50][112/217] loss = 0.61, accuracy = 60.34
[31/50][113/217] loss = 0.71, accuracy = 60.32
[31/50][114/217] loss = 0.65, accuracy = 60.31
[31/50][115/217] loss = 0.66, accuracy = 60.29
[31/50][116/217] loss = 0.64, accuracy = 60.31
[31/50][117/217] loss = 0.68, accuracy = 60.26
[31/50][118/217] loss = 0.63, accuracy = 60.28
[31/50][119/217] loss = 0.64, accuracy = 60.26
[31/50][120/217] loss = 0.66, accuracy = 60.21
[31/50][121/217] loss = 0.59, accuracy = 60.28
[31/50][122/217] loss = 0.66, accuracy = 60.23
[31/50][123/2

[32/50][63/217] loss = 0.69, accuracy = 59.86
[32/50][64/217] loss = 0.62, accuracy = 59.88
[32/50][65/217] loss = 0.60, accuracy = 59.97
[32/50][66/217] loss = 0.67, accuracy = 59.93
[32/50][67/217] loss = 0.63, accuracy = 59.93
[32/50][68/217] loss = 0.66, accuracy = 59.90
[32/50][69/217] loss = 0.64, accuracy = 59.96
[32/50][70/217] loss = 0.63, accuracy = 60.04
[32/50][71/217] loss = 0.63, accuracy = 60.05
[32/50][72/217] loss = 0.68, accuracy = 60.08
[32/50][73/217] loss = 0.63, accuracy = 60.14
[32/50][74/217] loss = 0.63, accuracy = 60.19
[32/50][75/217] loss = 0.64, accuracy = 60.18
[32/50][76/217] loss = 0.63, accuracy = 60.25
[32/50][77/217] loss = 0.63, accuracy = 60.28
[32/50][78/217] loss = 0.67, accuracy = 60.30
[32/50][79/217] loss = 0.63, accuracy = 60.31
[32/50][80/217] loss = 0.63, accuracy = 60.32
[32/50][81/217] loss = 0.67, accuracy = 60.33
[32/50][82/217] loss = 0.63, accuracy = 60.34
[32/50][83/217] loss = 0.66, accuracy = 60.29
[32/50][84/217] loss = 0.64, accur

In [ ]:


if not pretrained:
    train_attacker(attack_net_nn, shadow_net, shadow_train_loader, shadow_out_loader, attack_optim_nn, 
                   attack_loss, n_epochs=n_epochs_attack, k=k)
    
else:
    fnm = 'model_weights/CNN_voice_classifier'+str(data_set)+'_attack_'+transform_type+str(24)+'.pth'
#     fnm = 'model_weights/CNN_voice_classifier100_target_'+str(epoch)+'.pth'
    load_checkpoint(model = attack_net_nn, optimizer = attack_optim_nn, checkpoint = fnm)
#     chpt = torch.load(fnm)
#     attack_net_nn.load_state_dict(chpt['state_dict'])

#original:
fn = 'model_weights/CNN_voice_classifier'+str(data_set)+'_shadow_'+transform_type+str(n_epochs-1)+'.pth'

if not pretrained:

    models.save_checkpoint(model = shadow_net, optimizer = shadow_optim,
                           epoch = n_epochs-1, data_descriptor = all_data, 
                           accuracy = [train_accuracy, test_accuracy],
                           filename = fn)# train_attacker(attack_net_nn, shadow_net, shadow_train_loader, shadow_out_loader, attack_optim_nn, attack_loss, n_epochs=50, k=k)

In [ ]:
#Train SVM
# shadow_svm.fit(shadowinputs, shadowtargets)

In [ ]:
fn = 'model_weights/CNN_voice_classifier'+str(data_set)+'_attack_'+transform_type+str(n_epochs-1)+'.pth'

if not pretrained:

    models.save_checkpoint(model = attack_net_nn, optimizer = attack_optim_nn,
                           epoch = n_epochs-1, data_descriptor = all_data, 
                           accuracy = [train_accuracy, test_accuracy],
                           filename = fn)

# Evaluate Attack Nets
How well the trained attack models classify a sample as in or out of a target model's training dataset, and how performance is affected by target hyperparameters and which models attack which targets.

In [ ]:
#attack net trained on svm shadow model on svm target, C=10, gamma=.1
# eval_attack_net(attack_net_svm, gen_svm, sv_target_train_loader, sv_target_out_loader, k)

In [ ]:
#attack net trained on svm shadow model on svm target, C=1, gamma=.01
# eval_attack_net(attack_net_svm, maxacc_svm, sv_target_train_loader, sv_target_out_loader, k)

In [ ]:
#attack net trained on nn shadow model on nn target
eval_attack_net(attack_net_nn, target_net, target_train_loader, target_out_loader, k)

In [ ]:
#attack net trained on nn shadow model on svm target, C=10, gamma=.1
# eval_attack_net(attack_net_nn, gen_svm, sv_target_train_loader, sv_target_out_loader, k)

In [ ]:

# do this for 10 & 100 speakers
# .2 S & 3 S
#sufficient training and over-training

#manual data: 

#Attack 1:
df.loc[len(df)] = ['MFCC',25,69.0,.9994,.9632,1,0.89,0.90] 
df.loc[len(df)] = ['MFCC',25,255.0,.9961,.9443,1,0.88,0.91] 
df.loc[len(df)] = ['STFT',25,69.0,0.9989,0.9451,1,0.89,0.92] 
df.loc[len(df)] = ['STFT',25,255.0,0.9958,0.9181,1,0.85,0.86] 

#Attack 3 w/max data: 
df.loc[len(df)] = ['STFT',25,139.0,.9985,.9073,3,.81,.90] 
df.loc[len(df)] = ['STFT',50,511.0,.9942,.9057,3,.84,.87] 
df.loc[len(df)] = ['MFCC',25,139.0,.9969,.9136,3,.82,.92] 
df.loc[len(df)] = ['MFCC',25,511.0,.9960,.9321,3,0.83,0.93]


#Attack 3 on Attack1 models:
df.loc[len(df)] = ['MFCC',25,69.0,.9994,.9632,3,0.84,0.95] 
df.loc[len(df)] = ['MFCC',25,255.0,.9961,.9443,3,0.84,0.94] 
df.loc[len(df)] = ['STFT',25,69.0,0.9989,0.9451,3,0.81,0.97] 
df.loc[len(df)] = ['STFT',25,255.0,0.9958,0.9181,3,0.81,0.90] 

df['Training epochs'] =df['Training epochs'].astype(float)
df['Attack type'] =df['Attack type'].astype(float)

#style table
import seaborn as sns

cg = sns.light_palette("green", as_cmap=True)
cm = sns.light_palette("magenta", as_cmap=True)
bl = sns.light_palette("blue", as_cmap=True)
orr = sns.light_palette("orange", as_cmap=True)
gr = sns.light_palette("gray", as_cmap=True)

# df.style.bar(subset=['Train accuracy', 'Test accuracy'], align='mid', color=['#d65f5f', '#5fba7d'])
s = df.style.\
    background_gradient(cmap=cg,subset=['Train accuracy', 'Test accuracy']).\
    background_gradient(cmap=bl,subset=['Precision', 'Recall']).\
    background_gradient(cmap=orr,subset=['Training epochs']).\
    background_gradient(cmap=gr,subset=['Attack type']).\
    background_gradient(cmap=cm,subset=['# speakers']).\
    format({"Train accuracy": "{:.2%}","Test accuracy": "{:.2%}"}).\
    hide_index().\
    set_properties(**{'font-size': "16pt",'column-size':"24pt",'width': '100px'})

s